# Pre-processing Choppers

When choppers are loaded from NeXus, they typically contain a number of fields that need to be processed before they can be used for computing wavelength ranges, etc.
This guide shows how to extract the relevant data from such a NeXus chopper and create a [scippneutron.chopper.DiskChopper](../../generated/modules/scippneutron.chopper.disk_chopper.DiskChopper.rst) object.

In [ ]:
import matplotlib.pyplot as plt
import plopp as pp
import scipp as sc

This guide uses fake data for a chopper which roughly represents what a real chopper loaded from NeXus looks like.
ScippNeutron has a function for generating this data:

In [ ]:
from scippneutron.data import chopper_mockup

chopper = chopper_mockup()
chopper

We can see that there is some information about the slits and geometry of the chopper as well as some timing-related data.
Take a look at the [NXdisk_chopper](https://manual.nexusformat.org/classes/base_classes/NXdisk_chopper.html) documentation for an overview of the fields.
In this case, there already is a `position`.
This typically needs to be computed first, see [scippnexus.compute_positions](https://scipp.github.io/scippnexus/generated/functions/scippnexus.compute_positions.html).

Some fields are nested data groups which appear when a NeXus file contains `NXlog`s.
We can extract the relevant arrays from them using `post_process_disk_chopper`:

In [ ]:
from scippneutron.chopper import post_process_disk_chopper

chopper = post_process_disk_chopper(chopper)
chopper

Some data varies with time.
This is difficult to deal with, e.g., when [unwrapping frames](./frame-unwrapping.ipynb).
So we need to extract time-independent quantities from the raw chopper data.

## Identify In-phase Regions

Frame unwrapping is only feasible when the chopper is in-phase with the neutron source as wavelength ranges and resolutions would change with time otherwise.
To identify regions where the chopper is in-phase, we first find plateaus in the `rotation_speed` which is the rotation frequency of the chopper.

In [ ]:
rotation_speed = chopper['rotation_speed']
rotation_speed.name = 'rotation_speed'
rotation_speed

The chopper has a long region of near-constant rotation speed surrounded by spin-up and spin-down regions:

In [ ]:
rotation_speed.plot(markersize=2)

Use [find_plateaus](../../generated/modules/scippneutron.chopper.filtering.collapse_plateaus.rst) and [collapse_plateaus](../../generated/modules/scippneutron.chopper.filtering.collapse_plateaus.rst) to find those plateaus.
Note the `atol` and `min_n_points` parameters which need to be tuned for specific data.

In [ ]:
from scippneutron.chopper import collapse_plateaus, find_plateaus

plateaus = find_plateaus(rotation_speed, atol=sc.scalar(1e-3, unit='Hz'), min_n_points=10)
plateaus = collapse_plateaus(plateaus)
plateaus

In this case, the `find_plateaus` found two plateaus that we can plot with the following helper function:

In [ ]:
def plot_plateaus(raw_data: sc.DataArray, plateaus: sc.DataArray) -> None:
    fig, ax = plt.subplots(1)
    raw_data.plot(ax=ax, markersize=2)
    for plateau in plateaus:
        i = plateau.coords['plateau'].value
        da = sc.DataArray(
            plateau.data.broadcast(dims=['time'], shape=[2]),
            coords={'time': plateau.coords['time']},
            name=f'Plateau {i}')
        da.plot(ax=ax, ls='-', marker='|', c=f'C{i + 1}')

In [ ]:
plot_plateaus(rotation_speed, plateaus)

In this case, the source has a frequency of 14Hz which means that plateau 0 is in phase.
But plateau 1 is not, it is a short region where the chopper slowed down before fully stopping.

We can use [filter_in_phase](../../generated/modules/scippneutron.chopper.filtering.filter_in_phase.rst) to remove all out-of-phase plateaus:

In [ ]:
source_frequency = sc.scalar(14.0, unit='Hz')

In [ ]:
from scippneutron.chopper import filter_in_phase

frequency_in_phase = filter_in_phase(plateaus,
                           reference=source_frequency,
                           rtol=sc.scalar(1e-3))
frequency_in_phase

In [ ]:
plot_plateaus(rotation_speed, frequency_in_phase)

## Extract Plateau

Since there is only one plateau left, we can simply index into it to get the chopper frequency:

In [ ]:
frequency = frequency_in_phase['plateau', 0]
frequency

Next, we need the TDC timestamps for the in-phase region.
We can do this with label-based-indexing but need to construct a helper data array first:

In [ ]:
tdc = chopper['top_dead_center']
tdc

In [ ]:
aux = sc.DataArray(sc.scalar(0).broadcast(sizes=tdc.sizes),
                   coords={'time': tdc})
low = frequency.coords['time'][0]
high = frequency.coords['time'][1]
tdc_in_phase = aux['time', low:high].coords['time']
tdc_in_phase

We can check that the rate at which the TDC triggers is indeed close to 14Hz.
(The deviation comes from the way the fake data is constructed.)

In [ ]:
diff = tdc_in_phase[1:] - tdc_in_phase[:-1]
rate = 1 / diff.to(unit='s', dtype='float64')
rate.min(), rate.max()

## Compute Chopper Phase

## Build `DiskChopper`

Finally, we can assemble all data into a [scippneutron.chopper.DiskChopper](../../generated/modules/scippneutron.chopper.disk_chopper.DiskChopper.rst) object:

In [ ]:
processed = chopper.copy()
processed['rotation_speed'] = frequency.data
processed['phase'] = ...

The input data does not contain a beam position (the angle between the beam and TDC).
This probably means that it is 0.
But since `DiskChopper` does not make that assumption we have to be explicit:

In [ ]:
processed['beam_position'] = sc.scalar(0.0, unit='rad')

In [ ]:
from scippneutron.chopper import DiskChopper

disk_chopper = DiskChopper.from_nexus(processed)
disk_chopper